In [75]:
import pandas as pd
import numpy as np

# Load the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Add TAVG column with NaN values to test dataframe
test['TAVG'] = np.nan

# Combine the rows of test and train
combined = pd.concat([train, test], ignore_index=True)

# Convert DATE into day, month, and year columns
combined['DATE'] = pd.to_datetime(combined['DATE'],dayfirst=True)
combined['Day'] = combined['DATE'].dt.day
combined['Month'] = combined['DATE'].dt.month
combined['Year'] = combined['DATE'].dt.year

# Sort rows in ascending order based on date
combined = combined.sort_values(by='DATE')

# Find the rows where PRCP_A has missing values and set its value to 8% of PRCP_B
combined.loc[combined['PRCP_A'].isna(), 'PRCP_A'] = 0.08 * combined['PRCP_B']

# Find the rows where PRCP_C has missing values and set its value to 8% of PRCP_B
combined.loc[combined['PRCP_C'].isna(), 'PRCP_C'] = 0.08 * combined['PRCP_B']

# Sort rows based on the month
combined = combined.sort_values(by='Month')


# Create another column MAVG_A whose rows have value 1 wherever TAVG_A was missing
combined['MAVG_A'] = np.where(combined['TAVG_A'].isna(), 1, 0)
# Create another column MAVG_A whose rows have value 1 wherever TAVG_A was missing
combined['MAVG_B'] = np.where(combined['TAVG_B'].isna(), 1, 0)
# Create another column MAVG_A whose rows have value 1 wherever TAVG_A was missing
combined['MAVG_C'] = np.where(combined['TAVG_C'].isna(), 1, 0)
# Fill missing TAVG_A values with the average value of the corresponding month's TAVG_A
combined['TAVG_A'] = combined.groupby('Month')['TAVG_A'].transform(lambda x: x.fillna(x.mean()))
# Fill missing TAVG_B values with the average value of the corresponding month's TAVG_B
combined['TAVG_B'] = combined.groupby('Month')['TAVG_B'].transform(lambda x: x.fillna(x.mean()))
# Fill missing TAVG_C values with the average value of the corresponding month's TAVG_C
combined['TAVG_C'] = combined.groupby('Month')['TAVG_C'].transform(lambda x: x.fillna(x.mean()))

# Create another column MAVG_A whose rows have value 1 wherever TAVG_A was missing
combined['MMAX_A'] = np.where(combined['TMAX_A'].isna(), 1, 0)
# Create another column MAVG_A whose rows have value 1 wherever TAVG_A was missing
combined['MMAX_B'] = np.where(combined['TMAX_B'].isna(), 1, 0)
# Create another column MAVG_A whose rows have value 1 wherever TAVG_A was missing
combined['MMAX_C'] = np.where(combined['TMAX_C'].isna(), 1, 0)
# Fill missing TMAX_A values with the average value of the corresponding month's TMAX_A
combined['TMAX_A'] = combined.groupby('Month')['TMAX_A'].transform(lambda x: x.fillna(x.mean()))
# Fill missing TMAX_B values with the average value of the corresponding month's TMAX_B
combined['TMAX_B'] = combined.groupby('Month')['TMAX_B'].transform(lambda x: x.fillna(x.mean()))
# Fill missing TMAX_C values with the average value of the corresponding month's TMAX_C
combined['TMAX_C'] = combined.groupby('Month')['TMAX_C'].transform(lambda x: x.fillna(x.mean()))


# Create another column MAVG_A whose rows have value 1 wherever TAVG_A was missing
combined['MMIN_A'] = np.where(combined['TMIN_A'].isna(), 1, 0)
# Create another column MAVG_A whose rows have value 1 wherever TAVG_A was missing
combined['MMIN_B'] = np.where(combined['TMIN_B'].isna(), 1, 0)
# Create another column MAVG_A whose rows have value 1 wherever TAVG_A was missing
combined['MMIN_C'] = np.where(combined['TMIN_C'].isna(), 1, 0)
# Fill missing TAVG_A values with the average value of the corresponding month's TAVG_A
combined['TMIN_A'] = combined.groupby('Month')['TMIN_A'].transform(lambda x: x.fillna(x.mean()))
# Fill missing TAVG_B values with the average value of the corresponding month's TAVG_B
combined['TMIN_B'] = combined.groupby('Month')['TMIN_B'].transform(lambda x: x.fillna(x.mean()))
# Fill missing TAVG_C values with the average value of the corresponding month's TAVG_C
combined['TMIN_C'] = combined.groupby('Month')['TMIN_C'].transform(lambda x: x.fillna(x.mean()))



# Find rows with non-zero PRCP_A and check if the average of TMAX_A, TMAX_B, TMAX_C is less than 0.5, if yes change PRCP_A to 0
mask_prcp = combined['PRCP_A'] != 0
avg_tmax = combined[['TMAX_A', 'TMAX_B', 'TMAX_C']].mean(axis=1)
combined.loc[mask_prcp & (avg_tmax < 0.5), 'PRCP_A'] = 0

mask_prcp = combined['PRCP_B'] != 0
avg_tmax = combined[['TMAX_A', 'TMAX_B', 'TMAX_C']].mean(axis=1)
combined.loc[mask_prcp & (avg_tmax < 0.5), 'PRCP_B'] = 0

mask_prcp = combined['PRCP_C'] != 0
avg_tmax = combined[['TMAX_A', 'TMAX_B', 'TMAX_C']].mean(axis=1)
combined.loc[mask_prcp & (avg_tmax < 0.5), 'PRCP_C'] = 0



# Find rows with non-zero SNWD_A and check if the average of TMIN_A, TMIN_B, TMIN_C is more than 0.5, if yes change SNWD_A to 0
mask_snwd = combined['SNWD_A'] != 0
avg_tmin = combined[['TMIN_A', 'TMIN_B', 'TMIN_C']].mean(axis=1)
combined.loc[mask_snwd & (avg_tmin > 0.5), 'SNWD_A'] = 0

mask_snwd = combined['SNWD_B'] != 0
avg_tmin = combined[['TMIN_A', 'TMIN_B', 'TMIN_C']].mean(axis=1)
combined.loc[mask_snwd & (avg_tmin > 0.5), 'SNWD_B'] = 0

mask_snwd = combined['SNWD_C'] != 0
avg_tmin = combined[['TMIN_A', 'TMIN_B', 'TMIN_C']].mean(axis=1)
combined.loc[mask_snwd & (avg_tmin > 0.5), 'SNWD_C'] = 0

# Save the result to a new file
combined.to_csv('processed_combined_two.csv', index=False)


In [84]:
import pandas as pd
import numpy as np

# Read the CSV data
df = pd.read_csv('processed_combined_two.csv')



# Calculate the 3-day moving average for TAVG_A
df['TMAX_A_MA'] = df['TMAX_A'].rolling(window=3, center=True).mean()

# Function to identify and fix outliers using moving average method
def fix_outliers(row):
    if pd.notnull(row['TMAX_A_MA']):
        diff = abs(row['TMAX_A'] - row['TMAX_A_MA'])
        threshold = 12  # You can adjust this threshold as needed

        if diff > threshold:
            return (row['TMAX_B'] + row['TMAX_C']) / 2

    return row['TMAX_A']

# Apply the outlier detection and correction
df['TMAX_A_CORRECTED'] = df.apply(fix_outliers, axis=1)

# Replace the original TAVG_A with the corrected values
df['TMAX_A'] = df['TMAX_A_CORRECTED']

# Drop the temporary columns
df = df.drop(['TMAX_A_MA', 'TMAX_A_CORRECTED'], axis=1)




# Calculate the 3-day moving average for TAVG_A
df['TMAX_B_MA'] = df['TMAX_B'].rolling(window=3, center=True).mean()

# Function to identify and fix outliers using moving average method
def fix_outliers(row):
    if pd.notnull(row['TMAX_B_MA']):
        diff = abs(row['TMAX_B'] - row['TMAX_B_MA'])
        threshold = 12  # You can adjust this threshold as needed

        if diff > threshold:
            return (row['TMAX_A'] + row['TMAX_C']) / 2

    return row['TMAX_B']

# Apply the outlier detection and correction
df['TMAX_B_CORRECTED'] = df.apply(fix_outliers, axis=1)

# Replace the original TAVG_A with the corrected values
df['TMAX_B'] = df['TMAX_B_CORRECTED']

# Drop the temporary columns
df = df.drop(['TMAX_B_MA', 'TMAX_B_CORRECTED'], axis=1)







# Calculate the 3-day moving average for TAVG_A
df['TMAX_C_MA'] = df['TMAX_C'].rolling(window=3, center=True).mean()

# Function to identify and fix outliers using moving average method
def fix_outliers(row):
    if pd.notnull(row['TMAX_C_MA']):
        diff = abs(row['TMAX_C'] - row['TMAX_C_MA'])
        threshold = 12  # You can adjust this threshold as needed

        if diff > threshold:
            return (row['TMAX_A'] + row['TMAX_B']) / 2

    return row['TMAX_C']

# Apply the outlier detection and correction
df['TMAX_C_CORRECTED'] = df.apply(fix_outliers, axis=1)

# Replace the original TAVG_A with the corrected values
df['TMAX_C'] = df['TMAX_C_CORRECTED']

# Drop the temporary columns
df = df.drop(['TMAX_C_MA', 'TMAX_C_CORRECTED'], axis=1)


#___________________________________________________________________________________________________________________________________________________

# Calculate the 3-day moving average for TAVG_A
df['TAVG_A_MA'] = df['TAVG_A'].rolling(window=3, center=True).mean()

# Function to identify and fix outliers using moving average method
def fix_outliers(row):
    if pd.notnull(row['TAVG_A_MA']):
        diff = abs(row['TAVG_A'] - row['TAVG_A_MA'])
        threshold = 12  # You can adjust this threshold as needed

        if diff > threshold:
            return (row['TAVG_B'] + row['TAVG_C']) / 2

    return row['TAVG_A']

# Apply the outlier detection and correction
df['TAVG_A_CORRECTED'] = df.apply(fix_outliers, axis=1)

# Replace the original TAVG_A with the corrected values
df['TAVG_A'] = df['TAVG_A_CORRECTED']

# Drop the temporary columns
df = df.drop(['TAVG_A_MA', 'TAVG_A_CORRECTED'], axis=1)




# Calculate the 3-day moving average for TAVG_A
df['TAVG_B_MA'] = df['TAVG_B'].rolling(window=3, center=True).mean()

# Function to identify and fix outliers using moving average method
def fix_outliers(row):
    if pd.notnull(row['TAVG_B_MA']):
        diff = abs(row['TAVG_B'] - row['TAVG_B_MA'])
        threshold = 12  # You can adjust this threshold as needed

        if diff > threshold:
            return (row['TAVG_A'] + row['TAVG_C']) / 2

    return row['TAVG_B']

# Apply the outlier detection and correction
df['TAVG_B_CORRECTED'] = df.apply(fix_outliers, axis=1)

# Replace the original TAVG_A with the corrected values
df['TAVG_B'] = df['TAVG_B_CORRECTED']

# Drop the temporary columns
df = df.drop(['TAVG_B_MA', 'TAVG_B_CORRECTED'], axis=1)







# Calculate the 3-day moving average for TAVG_A
df['TAVG_C_MA'] = df['TAVG_C'].rolling(window=3, center=True).mean()

# Function to identify and fix outliers using moving average method
def fix_outliers(row):
    if pd.notnull(row['TAVG_C_MA']):
        diff = abs(row['TAVG_C'] - row['TAVG_C_MA'])
        threshold = 12  # You can adjust this threshold as needed

        if diff > threshold:
            return (row['TAVG_A'] + row['TAVG_B']) / 2

    return row['TAVG_C']

# Apply the outlier detection and correction
df['TAVG_C_CORRECTED'] = df.apply(fix_outliers, axis=1)

# Replace the original TAVG_A with the corrected values
df['TAVG_C'] = df['TAVG_C_CORRECTED']

# Drop the temporary columns
df = df.drop(['TAVG_C_MA', 'TAVG_C_CORRECTED'], axis=1)


#___________________________________________________________________________________________________________________________________________________________


# Calculate the 3-day moving average for TAVG_A
df['TMIN_A_MA'] = df['TMIN_A'].rolling(window=3, center=True).mean()

# Function to identify and fix outliers using moving average method
def fix_outliers(row):
    if pd.notnull(row['TMIN_A_MA']):
        diff = abs(row['TMIN_A'] - row['TMIN_A_MA'])
        threshold = 12  # You can adjust this threshold as needed

        if diff > threshold:
            return (row['TMIN_B'] + row['TMIN_C']) / 2

    return row['TMIN_A']

# Apply the outlier detection and correction
df['TMIN_A_CORRECTED'] = df.apply(fix_outliers, axis=1)

# Replace the original TAVG_A with the corrected values
df['TMIN_A'] = df['TMIN_A_CORRECTED']

# Drop the temporary columns
df = df.drop(['TMIN_A_MA', 'TMIN_A_CORRECTED'], axis=1)




# Calculate the 3-day moving average for TAVG_A
df['TMIN_B_MA'] = df['TMIN_B'].rolling(window=3, center=True).mean()

# Function to identify and fix outliers using moving average method
def fix_outliers(row):
    if pd.notnull(row['TMIN_B_MA']):
        diff = abs(row['TMIN_B'] - row['TMIN_B_MA'])
        threshold = 12  # You can adjust this threshold as needed

        if diff > threshold:
            return (row['TMIN_A'] + row['TMIN_C']) / 2

    return row['TMIN_B']

# Apply the outlier detection and correction
df['TMIN_B_CORRECTED'] = df.apply(fix_outliers, axis=1)

# Replace the original TAVG_A with the corrected values
df['TMIN_B'] = df['TMIN_B_CORRECTED']

# Drop the temporary columns
df = df.drop(['TMIN_B_MA', 'TMIN_B_CORRECTED'], axis=1)







# Calculate the 3-day moving average for TAVG_A
df['TMIN_C_MA'] = df['TMIN_C'].rolling(window=3, center=True).mean()

# Function to identify and fix outliers using moving average method
def fix_outliers(row):
    if pd.notnull(row['TMIN_C_MA']):
        diff = abs(row['TMIN_C'] - row['TMIN_C_MA'])
        threshold = 12  # You can adjust this threshold as needed

        if diff > threshold:
            return (row['TMIN_A'] + row['TMIN_B']) / 2

    return row['TMIN_C']

# Apply the outlier detection and correction
df['TMIN_C_CORRECTED'] = df.apply(fix_outliers, axis=1)

# Replace the original TAVG_A with the corrected values
df['TMIN_C'] = df['TMIN_C_CORRECTED']

# Drop the temporary columns
df = df.drop(['TMIN_C_MA', 'TMIN_C_CORRECTED'], axis=1)












# Save the corrected data
df.to_csv('yamma_yamma_yamma.csv', index=False)

print("Outliers have been identified and corrected using the moving average method. The updated data has been saved to 'processed_combined_two_corrected.csv'.")

Outliers have been identified and corrected using the moving average method. The updated data has been saved to 'processed_combined_two_corrected.csv'.


In [89]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = 'yamma_yamma_yamma.csv'
df = pd.read_csv(file_path)

# Convert the date column to datetime format if not already
df['DATE'] = pd.to_datetime(df['DATE'])

# Sort the dataframe based on dates in ascending order
df = df.sort_values(by='DATE').reset_index(drop=True)

# Find rows where TAVG is NaN
missing_tavg_indices = df[df['TAVG'].isna()].index

# Function to find nearest non-NaN TAVG and calculate the difference in days
def find_nearest_tavg(index, df):
    current_date = df.loc[index, 'DATE']

    # Get all non-NaN TAVG rows
    non_nan_tavg_rows = df.dropna(subset=['TAVG'])

    # Calculate the absolute difference in days
    non_nan_tavg_rows['DAY_DIFF'] = (non_nan_tavg_rows['DATE'] - current_date).abs()

    # Get the row with the minimum day difference
    nearest_row = non_nan_tavg_rows.loc[non_nan_tavg_rows['DAY_DIFF'].idxmin()]

    adj_tavg = nearest_row['TAVG']
    diff_days = (nearest_row['DATE'] - current_date).days

    return adj_tavg, diff_days

# Apply the function to each missing TAVG row
df.loc[missing_tavg_indices, 'ADJ_TAVG'] = missing_tavg_indices.map(lambda x: find_nearest_tavg(x, df)[0])
df.loc[missing_tavg_indices, 'DIFF'] = missing_tavg_indices.map(lambda x: find_nearest_tavg(x, df)[1])

# Remove all rows where TAVG is not NaN
df = df[df['TAVG'].isna()]

df = df.drop(columns=['MAVG_A','MMAX_A','MMIN_A','TAVG'])
df = df.drop(columns=['MAVG_B','MMAX_B','MMIN_B','Year'])
df = df.drop(columns=['MAVG_C','MMAX_C','MMIN_C','Day'])
df = df.drop(columns=['INDEX'])
df = df.drop(columns=['LATITUDE_A','LONGITUDE_A','ELEVATION_A'])
df = df.drop(columns=['LATITUDE_B','LONGITUDE_B','ELEVATION_B'])
df = df.drop(columns=['LATITUDE_C','LONGITUDE_C','ELEVATION_C'])
df = df.drop(columns=['LATITUDE','LONGITUDE'])



# Save the modified dataframe to a new CSV file
output_file_path = 'cleaned_TestingSet.csv'
df.to_csv(output_file_path, index=False)


<ipython-input-89-fdcbde96c430>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_nan_tavg_rows['DAY_DIFF'] = (non_nan_tavg_rows['DATE'] - current_date).abs()
<ipython-input-89-fdcbde96c430>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_nan_tavg_rows['DAY_DIFF'] = (non_nan_tavg_rows['DATE'] - current_date).abs()
<ipython-input-89-fdcbde96c430>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [96]:
import pandas as pd

# Load the datasets
test_df = pd.read_csv('test.csv')
cleaned_testing_df = pd.read_csv('cleaned_TestingSet.csv')

# Ensure the DATE column is in datetime format
test_df['DATE'] = pd.to_datetime(test_df['DATE'],dayfirst=True)
cleaned_testing_df['DATE'] = pd.to_datetime(cleaned_testing_df['DATE'])

# Sort the cleaned_TestingSet based on the order of the DATE column in the test dataset
sorted_cleaned_testing_df = cleaned_testing_df.set_index('DATE').loc[test_df['DATE']].reset_index()
sorted_cleaned_testing_df['DIFF'] = sorted_cleaned_testing_df['DIFF'].abs()
# Normalize the DIFF column to values between 0 and 1
#diff_min = sorted_cleaned_testing_df['DIFF'].min()
diff_max = sorted_cleaned_testing_df['DIFF'].max()
sorted_cleaned_testing_df['DIFF'] = (sorted_cleaned_testing_df['DIFF'] ) / (diff_max)


# Save the sorted dataframe
sorted_cleaned_testing_df.to_csv('sorted_TestingSet.csv', index=False)


In [97]:
import pandas as pd
import numpy as np

# Load the CSV file
file_path = 'yamma_yamma_yamma.csv'
df = pd.read_csv(file_path)

# Step 1: Remove rows with NaN values in the 'TAVG' column
df = df.dropna(subset=['TAVG'])

# Step 2: Sort the rows in ascending order based on dates
df = df.sort_values(by=['DATE'])

# Step 3: Calculate the ADJ_TAVG and DIFF columns
adj_tavg_list = []
diff_list = []

for i in range(len(df)):
    current_date = pd.to_datetime(df.iloc[i]['DATE'])
    current_tavg = df.iloc[i]['TAVG']

    # Calculate differences in days for all other dates
    df['DATE_DIFF'] = (pd.to_datetime(df['DATE']) - current_date).abs().dt.days

    # Find the closest date that is not the same row (excluding zero-day difference)
    closest_row = df.loc[df['DATE_DIFF'] > 0].iloc[df['DATE_DIFF'].idxmin()]

    adj_tavg_list.append(closest_row['TAVG'])
    diff_list.append(closest_row['DATE_DIFF'])

# Add ADJ_TAVG and DIFF columns to the DataFrame
df['ADJ_TAVG'] = adj_tavg_list
df['DIFF'] = diff_list

# Step 4: Shuffle (unsort) the rows
df = df.sample(frac=1).reset_index(drop=True)


df = df.drop(columns=['MAVG_A','MMAX_A','MMIN_A','DATE'])
df = df.drop(columns=['MAVG_B','MMAX_B','MMIN_B','Year'])
df = df.drop(columns=['MAVG_C','MMAX_C','MMIN_C','Day'])
df = df.drop(columns=['INDEX'])
df = df.drop(columns=['LATITUDE_A','LONGITUDE_A','ELEVATION_A'])
df = df.drop(columns=['LATITUDE_B','LONGITUDE_B','ELEVATION_B'])
df = df.drop(columns=['LATITUDE_C','LONGITUDE_C','ELEVATION_C'])
df = df.drop(columns=['LATITUDE','LONGITUDE'])

df['DIFF'] = df['DIFF'].abs()
# Normalize the DIFF column to values between 0 and 1
#diff_min = sorted_cleaned_testing_df['DIFF'].min()
diff_max = df['DIFF'].max()
df['DIFF'] = (df['DIFF'] ) / (diff_max)



# Step 5: Save the modified DataFrame to a new CSV file
output_file_path = 'modified_train.csv'
df.to_csv(output_file_path, index=False)


IndexError: single positional indexer is out-of-bounds

In [102]:
import pandas as pd
import numpy as np

# Load the CSV file
df = pd.read_csv('yamma_yamma_yamma.csv')

# Remove rows with NaN TAVG values
df = df.dropna(subset=['TAVG'])

# Convert the date column to datetime format if it's not already
df['DATE'] = pd.to_datetime(df['DATE'])

# Sort the rows in ascending order based on dates
df = df.sort_values(by='DATE').reset_index(drop=True)

# Initialize the new columns
df['ADJ_TAVG'] = np.nan
df['DIFF'] = np.nan

# Iterate over each row to find the closest date
for i in range(len(df)):
    # Exclude the current row to avoid matching with itself
    temp_df = df.drop(i)

    # Find the closest date and the corresponding TAVG
    closest_row = temp_df.iloc[(temp_df['DATE'] - df.loc[i, 'DATE']).abs().argsort()[:1]]
    df.loc[i, 'ADJ_TAVG'] = closest_row['TAVG'].values[0]
    df.loc[i, 'DIFF'] = (closest_row['DATE'].values[0] - df.loc[i, 'DATE']).days

# Shuffle the rows (unsort)
df = df.sample(frac=1).reset_index(drop=True)


df = df.drop(columns=['MAVG_A','MMAX_A','MMIN_A','DATE'])
df = df.drop(columns=['MAVG_B','MMAX_B','MMIN_B','Year'])
df = df.drop(columns=['MAVG_C','MMAX_C','MMIN_C','Day'])
df = df.drop(columns=['INDEX'])
df = df.drop(columns=['LATITUDE_A','LONGITUDE_A','ELEVATION_A'])
df = df.drop(columns=['LATITUDE_B','LONGITUDE_B','ELEVATION_B'])
df = df.drop(columns=['LATITUDE_C','LONGITUDE_C','ELEVATION_C'])
df = df.drop(columns=['LATITUDE','LONGITUDE'])

df['DIFF'] = df['DIFF'].abs()
# Normalize the DIFF column to values between 0 and 1
#diff_min = sorted_cleaned_testing_df['DIFF'].min()
diff_max = df['DIFF'].max()
df['DIFF'] = (df['DIFF'] ) / (diff_max)

df = df.drop(columns=['DIFF'])

# Save the modified dataframe to a new CSV file
df.to_csv('fina_train.csv', index=False)
